In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np

# 訓練データのCSVファイルを読み込む
train_df = pd.read_csv('drive/MyDrive/心不全予測/train.csv')

# 特徴量とターゲット変数にデータを分割
X_train = train_df.drop(['id', 'target'], axis=1)
y_train = train_df['target']

# データを訓練用と検証用に分割
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# XGBoost用のデータセットを作成
dtrain_split = xgb.DMatrix(X_train_split, label=y_train_split)
dval = xgb.DMatrix(X_val, label=y_val)

# ハイパーパラメータの空間を定義
param_space = {
    'max_depth': np.arange(1, 11),  # 1から10までの整数
    'n_estimators': np.arange(100, 1000, 100)  # 100から900までの100刻みの整数
}

# ランダムにハイパーパラメータを選択
random_params = {
    'max_depth': np.random.choice(param_space['max_depth']),
    'n_estimators': np.random.choice(param_space['n_estimators'])
}

# モデルのトレーニング
num_round = 100
evals_result = {}
bst_random = xgb.train(random_params, dtrain_split, num_round, evals=[(dval, 'validation')],
                       evals_result=evals_result, early_stopping_rounds=10)

# 検証精度を表示
validation_error = evals_result['validation']['error']
print("Validation error:", validation_error[-1])



In [ ]:

# テストデータのCSVファイルを読み込む
test_df = pd.read_csv('drive/MyDrive/心不全予測/test.csv')

# テストデータから特徴量を取得
X_test = test_df.drop('id', axis=1)

# XGBoost用のデータセットを作成
dtest = xgb.DMatrix(X_test)

# テストデータの予測
y_pred_proba_random = bst_random.predict(dtest)
y_pred_random = [1 if p >= 0.5 else 0 for p in y_pred_proba_random]

# 予測結果を持つDataFrameを作成
submission_df_random = pd.DataFrame({
    'index': test_df['id'],  # テストデータのインデックス
    'prediction': y_pred_random        # 予測結果
})

# CSVファイルとして保存（ヘッダ無し）
submission_df_random.to_csv('drive/MyDrive/心不全予測/predictions_random.csv', header=False, index=False)